# https://github.com/aws-samples/amazon-sagemaker-immersion-day/blob/lab03/bring-custom-container.ipynb

In [ ]:
!curl https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-immersion-day/master/scikit_bring_your_own.zip --output scikit_bring_your_own.zip

In [ ]:
!unzip scikit_bring_your_own.zip

In [ ]:
!mv scikit_bring_your_own/data ./

In [ ]:
!mv scikit_bring_your_own/container ./

In [ ]:
!rm scikit_bring_your_own.zip

In [ ]:
!rm -rf scikit_bring_your_own

In [ ]:
!cat container/Dockerfile

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
%%sh

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

sm-docker build .

In [ ]:
# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

iamgeuri = '054898670646.dkr.ecr.us-east-2.amazonaws.com/sagemaker-studio-d-5um3hlokgoth:default-1673743370029'
tree = sage.estimator.Estimator(iamgeuri,
                       role, instance_count=1, instance_type='ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

In [ ]:
from sagemaker.serializers import CSVSerializer
predictor = tree.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=CSVSerializer())

In [ ]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

In [ ]:
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

In [ ]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

In [ ]:
print(predictor.predict(test_data.values).decode('utf-8'))

In [ ]:
predictor.endpoint_name

In [ ]:
sess.delete_endpoint(predictor.endpoint_name)